In [1]:
import torch
import torchvision.models as models
from torchvision import datasets, transforms
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter


In [2]:
# train_dataset = CustomDataset()
# val_dataset = CustomDataset()

# train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
# val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [18]:
# Define the transformations to apply to the CIFAR-10 data
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

download = True
# Define the training and test datasets
train_dataset = datasets.Flowers102(root='./data', split="train", download=download, transform=data_transforms)
test_dataset = datasets.Flowers102(root='./data', split="test", download=download, transform=data_transforms)

# Define the dataloaders to load the data in batches during training and testing
batch_size = 64

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)


In [4]:
print(f"Number of training batches: {len(train_loader)}")
print(f"Number of testing batches: {len(test_loader)}")

Number of training batches: 16
Number of testing batches: 97


In [19]:
# the network to test
model = models.resnet18(pretrained=False)
num_classes = 102

learning_rate = 1e-3

# Modify the last fully connected layer
fc_input = model.fc.in_features
model.fc = nn.Linear(fc_input, num_classes)

# print(model)

# Step 5: Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [6]:
num_params = sum(torch.numel(p) for p in model.parameters())
print(f"Number of parameters: {num_params}")

Number of parameters: 11228838


In [7]:
log_dir = "./logs/flowers"  # Set the directory for storing the logs
writer = SummaryWriter(log_dir)

In [8]:
def model_eval(model, dataloader):
    
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in dataloader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    return accuracy

In [21]:
num_epochs = 100
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

log_train_every = 1
log_test_every = 5

for epoch in range(num_epochs):
    running_loss = 0.0
    
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

#         if (i+1) % 100 == 0:
#             print(f"[Epoch: {epoch + 1}, Batch: {i + 1}] Loss: {running_loss/100:.3f}")
#             running_loss = 0.0
        
    # compute training & testing accuracy every couple of iterations        
    if (epoch+1) % log_train_every == 0:
        train_accuracy = model_eval(model, train_loader)

        # Log the loss
        #writer.add_scalar('Loss/train', loss.cpu().item(), epoch * len(train_loader) + i)

        # Log the training accuracy
        #writer.add_scalar('Accuracy/train', train_accuracy, epoch * len(train_loader) + i)
        print(f"Epoch {epoch+1} : Loss: {running_loss} Accuracy: {train_accuracy}")
        
    if (epoch+1) % log_test_every == 0:
        test_accuracy = model_eval(model, test_loader)

        # Log the test accuracy
        #writer.add_scalar('Accuracy/test', test_accuracy, epoch * len(train_loader) + i)
        print(f"Epoch {epoch+1} : Test Accuracy: {test_accuracy}")
        
writer.close()
print("Training finished.")

Epoch 1 : Loss: 71.62561893463135 Accuracy: 3.4313725490196076
Epoch 2 : Loss: 83.44756746292114 Accuracy: 2.450980392156863
Epoch 3 : Loss: 72.786630153656 Accuracy: 2.156862745098039
Epoch 4 : Loss: 70.43598651885986 Accuracy: 3.5294117647058822
Epoch 5 : Loss: 67.73175287246704 Accuracy: 3.627450980392157
Epoch 5 : Test Accuracy: 3.2850870060172386
Epoch 6 : Loss: 65.91990351676941 Accuracy: 6.176470588235294
Epoch 7 : Loss: 64.32340049743652 Accuracy: 6.862745098039215
Epoch 8 : Loss: 63.408204793930054 Accuracy: 5.490196078431373
Epoch 9 : Loss: 62.26842260360718 Accuracy: 9.019607843137255
Epoch 10 : Loss: 61.68536925315857 Accuracy: 9.313725490196079
Epoch 10 : Test Accuracy: 7.350788746137583
Epoch 11 : Loss: 61.155158281326294 Accuracy: 8.235294117647058
Epoch 12 : Loss: 59.005356550216675 Accuracy: 11.176470588235293
Epoch 13 : Loss: 58.17576336860657 Accuracy: 11.372549019607844
Epoch 14 : Loss: 56.42585253715515 Accuracy: 15.882352941176471
Epoch 15 : Loss: 54.6535651683807

KeyboardInterrupt: 

## Curriculum Learning experiment 1

So.. The curriculum for the following training procedure is that we increase the number of classes progressively in the dataset. All original labels are shited by 1. So, label 0 becomes 1, 1 becomes 2.. and so on. 
We first start with 2 classes 0 and 1 where 0 represents rest of classes and 1 represents the class that was originally 0. Then we work with 3 classes after 2 epochs: 0(rest of the classes), 1 and 2.

In [14]:
class SubsetDataset(torch.utils.data.Dataset):
    def __init__(self, dataset, classes, num_classes, train=True, total_classes=102):
        self.dataset = dataset
        self.classes = classes
        self.num_classes = num_classes
        self.train = train
        self.total_classes = total_classes

    def __getitem__(self, index):
        image, label = self.dataset[index]
        if not self.train:
            return image, (label+1)%self.total_classes# we would reserve 0 for non-existing classes
        
        if label in self.classes[:self.num_classes]:
            return image, (label+1)%self.total_classes
        else:
            return torch.zeros_like(image), 0# IMPORTANT CHOICE MADE HERE 

    def __len__(self):
        return len(self.dataset)

In [15]:
num_epochs = 100
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

curriculum_epochs = 5
increment_step = 2
nclass = increment_step


log_train_every = 1
log_test_every = 5

test_dataset = SubsetDataset(train_dataset, list(range(num_classes)), num_classes, train=False)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, num_workers=4, shuffle=True)

for epoch in range(num_epochs):
    running_loss = 0.0
    
    subset_classes = list(range(nclass))

    # Create the modified dataset
    subset_dataset = SubsetDataset(train_dataset, subset_classes, nclass)

    # Create the train loader using the modified dataset
    train_loader = torch.utils.data.DataLoader(subset_dataset, batch_size=batch_size, num_workers=4, shuffle=True)

    
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

#         if (i+1) % 100 == 0:
#             print(f"[Epoch: {epoch + 1}, Batch: {i + 1}] Loss: {running_loss/100:.3f}")
#             running_loss = 0.0
        
    # compute training & testing accuracy every couple of iterations        
    if (epoch+1) % log_train_every == 0:
        train_accuracy = model_eval(model, train_loader)

        # Log the loss
        #writer.add_scalar('Loss/train', loss.cpu().item(), epoch * len(train_loader) + i)

        # Log the training accuracy
        #writer.add_scalar('Accuracy/train', train_accuracy, epoch * len(train_loader) + i)
        print(f"Epoch {epoch+1} : Loss: {running_loss} Accuracy: {train_accuracy}")
        
    if (epoch+1) % log_test_every == 0:
        test_accuracy = model_eval(model, test_loader)

        # Log the test accuracy
        #writer.add_scalar('Accuracy/test', test_accuracy, epoch * len(train_loader) + i)
        print(f"Epoch {epoch+1} : Test Accuracy: {test_accuracy}")
    
    nclass += increment_step

writer.close()
print("Training finished.")

Epoch 1 : Loss: 30.097966594155878 Accuracy: 99.01960784313725
Epoch 2 : Loss: 5.318315498530865 Accuracy: 96.96078431372548
Epoch 3 : Loss: 2.7389416117221117 Accuracy: 95.09803921568627
Epoch 4 : Loss: 3.874854639172554 Accuracy: 93.13725490196079
Epoch 5 : Loss: 4.6901357769966125 Accuracy: 91.17647058823529
Epoch 5 : Test Accuracy: 0.9803921568627451
Epoch 6 : Loss: 5.725368015468121 Accuracy: 90.0
Epoch 7 : Loss: 6.797103315591812 Accuracy: 88.33333333333333
Epoch 8 : Loss: 7.750844806432724 Accuracy: 85.98039215686275
Epoch 9 : Loss: 9.018923714756966 Accuracy: 84.2156862745098
Epoch 10 : Loss: 9.751175552606583 Accuracy: 81.96078431372548
Epoch 10 : Test Accuracy: 1.5686274509803921
Epoch 11 : Loss: 11.22835624217987 Accuracy: 80.68627450980392
Epoch 12 : Loss: 11.773661375045776 Accuracy: 79.01960784313725
Epoch 13 : Loss: 12.610673189163208 Accuracy: 78.13725490196079
Epoch 14 : Loss: 14.453419268131256 Accuracy: 75.0
Epoch 15 : Loss: 15.424874186515808 Accuracy: 74.2156862745

Process Process-572:
Exception ignored in: <function _releaseLock at 0x7fda3b5a43a0>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.9/logging/__init__.py", line 227, in _releaseLock
    def _releaseLock():
KeyboardInterrupt: 
Traceback (most recent call last):
  File "/opt/conda/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/opt/conda/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/asagrawal/private/.env/lib/python3.9/site-packages/torch/utils/data/_utils/worker.py", line 208, in _worker_loop
    def _worker_loop(dataset_kind, dataset, index_queue, data_queue, done_event,
KeyboardInterrupt


RuntimeError: DataLoader worker (pid(s) 58948, 59011) exited unexpectedly